In [1]:
test_chr={}
test_chr["chr3"]=1
test_chr["chr11"]=1

use_KO_filter=True
if use_KO_filter:
     KO_candidatefile = "/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2a/reditools2_candidates/HEK293T_DKO.candidate_sites.tab"
     KO_site_needtofilter={} #619
     input = open(KO_candidatefile)
     for line in input:
        coverage_KO=float(line.split("\t")[4])
        if coverage_KO>=10: #use 10 as coverage cutoff
            chr_=line.split("\t")[0]
            pos_=line.split("\t")[1]
            chrpos = chr_+"-"+pos_
            ratio =float(line.split("\t")[3])
            if ratio>0.05:
                KO_site_needtofilter[chrpos]=ratio


candidatefile="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2a/reditools2_candidates/HEK293T_WT.candidate_sites.tab"
shorread_coverage_cutoff=10#30#30 #100
longread_coverage_cutoff=6#30#30 #50
AG_ratio_per_site={} #101685 use_KO_filter=False  101386 use_KO_filter=True
input = open(candidatefile)
for line in input:
    chr_=line.split("\t")[0]
    pos_=line.split("\t")[1]
    chrpos = chr_+"-"+pos_
    if use_KO_filter:
       if chrpos in KO_site_needtofilter:#remove sites in KO>0.5 AGratio
             continue
    if float(line.split("\t")[4])<shorread_coverage_cutoff:
       continue

    ratio = float(line.split("\t")[3])
    if ratio < 0.1:
        continue

    AG_ratio_per_site[chrpos]=ratio


rediportal = "/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3a/REDIportal_hg38.txt"

REDIportal_sites={}
input = open(rediportal,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1]
    REDIportal_sites[chr_+"-"+pos_]=1





filename_pos="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3a/site_level_coverage/HEK293T_WT_directRNA_onlycandidate_cov5_ratio0_modcov0.sitelev.bed"

train_positive_output=open("train_HEK293T_WT_positive_sites.txt",'w')
test_positive_output=open("test_HEK293T_WT_positive_sites.txt",'w')

input=open(filename_pos)
train_num_candidate=0  #in our HEK WT after filter 24743
test_num_candidate=0
for line in input:
        readcoverage=int(line.split()[3])
        if readcoverage<longread_coverage_cutoff:
             continue

        chr_ = line.split()[0]
        pos_ = line.split()[1]

        if chr_+"-"+pos_ in AG_ratio_per_site:
            if chr_ in test_chr:
                  test_positive_output.write(chr_+"-"+pos_+"\n")
                  test_num_candidate+=1
            else:
                train_positive_output.write(chr_+"-"+pos_+"\n")
                train_num_candidate+=1
train_positive_output.close()
test_positive_output.close()
filename_neg="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3a/site_level_coverage/HEK293T_WT_directRNA_noncandidate_cov5_ratio0_modcov0.sitelev.bed"
train_negative_sites=[]
train_snp_sites=[]
test_negative_sites=[]
test_snp_sites=[]
input=open(filename_neg)
for line in input:
     readcoverage=int(line.split()[3])
     if readcoverage<longread_coverage_cutoff:
          continue

     chr_ = line.split()[0]
     pos_ = line.split()[1]
     if chr_+"-"+pos_ in AG_ratio_per_site:
          print("error !")
     else:
          if chr_+"-"+pos_  in REDIportal_sites:
               continue  #remove all sites in REDIportal_sites from neg set

          if chr_ in test_chr:
              test_negative_sites.append(chr_+"-"+pos_)
          else:
              train_negative_sites.append(chr_+"-"+pos_)

#extract balanced sites
import random
seed=1234

train_negative_output=open("train_HEK293T_WT_negative_sites.txt",'w')
random.Random(seed).shuffle(train_negative_sites)
for site in train_negative_sites[:train_num_candidate]:
    train_negative_output.write(site+"\n")
train_negative_output.close()

test_negative_output=open("test_HEK293T_WT_negative_sites.txt",'w')
random.Random(seed).shuffle(test_negative_sites)
for site in test_negative_sites[:test_num_candidate]:
    test_negative_output.write(site+"\n")
test_negative_output.close()



In [5]:
random.Random(seed).shuffle(test_negative_sites)